## File descriptions
* *sales_train.csv* - the training set. Daily historical data from January 2013 to October 2015.
* *test.csv* - the test set. You need to forecast the sales for these shops and products for November 2015.
* *sample_submission.csv* - a sample submission file in the correct format.
* *items.csv* - supplemental information about the items/products.
* *item_categories.csv*  - supplemental information about the items categories.
* *shops.csv* - supplemental information about the shops.


## Data fields
* *ID* - an Id that represents a (Shop, Item) tuple within the test set
* *shop_id* - unique identifier of a shop
* *item_id* - unique identifier of a product
* *item_category_id* - unique identifier of item category
* *item_cnt_day* - number of products sold. You are predicting a monthly amount of this measure
* *item_price* - current price of an item
* *date* - date in format dd/mm/yyyy
* *date_block_num* - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33
* *item_name* - name of item
* *shop_name* - name of shop
* *item_category_name* - name of item category


In [1]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from datetime import datetime, timedelta

#pd.set_option('display.float_format', lambda x: '%.5f' % x)

# Use 3 decimal places in output display
pd.set_option("display.precision", 3)


In [2]:
df_shops = pd.read_csv("shops.csv")
df_shops.describe()

,shop_id
count,60.000
mean,29.500
std,17.464
min,0.000
25%,14.750
50%,29.500
75%,44.250
max,59.000


In [3]:
df_categories = pd.read_csv("item_categories.csv")
df_categories.describe()

,item_category_id
count,84.000
mean,41.500
std,24.393
min,0.000
25%,20.750
50%,41.500
75%,62.250
max,83.000


In [4]:
df_items = pd.read_csv("items.csv")
df_items.describe()

,item_id,item_category_id
count,22170.000,22170.000
mean,11084.500,46.291
std,6400.072,15.941
min,0.000,0.000
25%,5542.250,37.000
50%,11084.500,40.000
75%,16626.750,58.000
max,22169.000,83.000


In [5]:
df_sales = pd.read_csv("sales_train.csv")
df_sales.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.936e+06,2.936e+06,2.936e+06,2.936e+06,2.936e+06
mean,1.457e+01,3.300e+01,1.020e+04,8.909e+02,1.243e+00
std,9.423e+00,1.623e+01,6.324e+03,1.730e+03,2.619e+00
min,0.000e+00,0.000e+00,0.000e+00,-1.000e+00,-2.200e+01
25%,7.000e+00,2.200e+01,4.476e+03,2.490e+02,1.000e+00
50%,1.400e+01,3.100e+01,9.343e+03,3.990e+02,1.000e+00
75%,2.300e+01,4.700e+01,1.568e+04,9.990e+02,1.000e+00
max,3.300e+01,5.900e+01,2.217e+04,3.080e+05,2.169e+03


In [6]:
df_sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [7]:
#df_sales.loc[(df_sales['date']=='13.01.2013') & df_sales['shop_id'].isin([0,1,57,58])]
#df_sales.loc[df_sales['shop_id'].isin([39, 40])]
#df_sales.loc[df_sales['shop_id'] == 40]
#df_sales.loc[~df_sales['item_id'].isin(df_items['item_id'].values)]
df_sales.loc[df_sales['item_cnt_day'] < 0]['shop_id'].value_counts()
#df_sales.loc[(df_sales['item_id'] == 11373) & (df_sales['shop_id'] == 5)]
#df_sales.loc[(df_sales['item_id'] == 11373)]['shop_id'].value_counts()

31    451
12    430
54    356
25    316
57    314
6     261
42    251
28    216
43    211
19    211
56    195
47    193
58    173
44    165
53    163
38    162
27    157
37    154
29    151
52    143
16    139
59    137
30    135
26    134
18    132
50    130
15    130
35    128
21    125
4     124
14    120
3     119
45    103
24    103
46    102
7      99
22     97
48     89
5      85
41     82
2      80
17     80
49     42
39     37
32     34
34     25
33     22
23     17
40     13
9       8
8       6
36      3
55      2
20      1
Name: shop_id, dtype: int64

In [8]:
#df_sales.groupby(['date_block_num','shop_id','item_id'])['item_cnt_day'].sum()
#df_mean = df_sales.groupby(['shop_id','item_id'])['item_cnt_day'].mean()
#df_mean.describe()

In [9]:
#df_shops.head(60)

In [10]:
df_test = pd.read_csv("test.csv")
df_test.describe()

,ID,shop_id,item_id
count,214200.000,214200.000,214200.000
mean,107099.500,31.643,11019.399
std,61834.358,17.562,6252.645
min,0.000,2.000,30.000
25%,53549.750,16.000,5381.500
50%,107099.500,34.500,11203.000
75%,160649.250,47.000,16071.500
max,214199.000,59.000,22167.000


In [11]:
valid_shops = np.sort(df_test['shop_id'].unique())
df_shops.loc[~df_shops['shop_id'].isin(valid_shops)] 

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
8,"Воронеж ТРЦ Сити-Парк ""Град""",8
9,Выездная Торговля,9
11,Жуковский ул. Чкалова 39м²,11
13,"Казань ТЦ ""Бехетле""",13
17,"Красноярск ТЦ ""Взлетка Плаза""",17
20,"Москва ""Распродажа""",20
23,"Москва ТК ""Буденовский"" (пав.А2)",23
27,"Москва ТЦ ""МЕГА Белая Дача II""",27


In [12]:
# get valid shop id
def valid_shop_id(id):
    if id == 0:
        return 57
    if id == 1:
        return 58
    if id == 23:
        return 24
    if id == 11:
        return 10
    if id == 40:
        return 39
    return id

def shop_type(shop):
    #types = ['ТК', 'ТЦ', 'ТРК', 'ТРЦ']
    p = re.compile(r"(Т[РКЦ]+)")
    r = p.search(shop)
    if r is not None: return r.group(1) 
    if 'нлайн' in shop: return 'Онлайн'
    if 'нтернет' in shop: return 'Онлайн'
    return 'Магазин'

# get city
def shop_city(shop):
    p = re.compile(r"^([а-яА-Я]*)")
    r = p.search(shop)
    if r is not None: return r.group(1) 
    return 'Unknown'

#df_shops['valid_id'] = df_shops['shop_id'] 

In [13]:
#print(shop_type('Выездная Торговля'))
print(shop_city('Москва ТЦ "МЕГА Белая Дача II"'))
print(shop_city('!Якутск Орджоникидзе, 56 фран'))

Москва



In [14]:
df_shops.drop([0,1], inplace=True)

In [15]:

df_shops['city'] = df_shops['shop_name'].apply(lambda x: shop_city(x))
df_shops['type'] = df_shops['shop_name'].apply(lambda x: shop_type(x))
df_shops['mega'] = df_shops['shop_name'].str.contains('мега', case=False).astype(int)

In [16]:
df_shops

,shop_name,shop_id,city,type,mega
2,"Адыгея ТЦ ""Мега""",2,Адыгея,ТЦ,1
3,"Балашиха ТРК ""Октябрь-Киномир""",3,Балашиха,ТРК,0
4,"Волжский ТЦ ""Волга Молл""",4,Волжский,ТЦ,0
5,"Вологда ТРЦ ""Мармелад""",5,Вологда,ТРЦ,0
6,"Воронеж (Плехановская, 13)",6,Воронеж,Магазин,0
7,"Воронеж ТРЦ ""Максимир""",7,Воронеж,ТРЦ,0
8,"Воронеж ТРЦ Сити-Парк ""Град""",8,Воронеж,ТРЦ,0
9,Выездная Торговля,9,Выездная,Магазин,0
10,Жуковский ул. Чкалова 39м?,10,Жуковский,Магазин,0
11,Жуковский ул. Чкалова 39м²,11,Жуковский,Магазин,0


In [17]:
df_categories['category'] = df_categories['item_category_name'].str.split('[-(]', n=0).str[0].str.strip()
df_categories['digital'] = df_categories['item_category_name'].str.contains('цифра', case=False).astype(int)

In [18]:
df_categories.head(10)

,item_category_name,item_category_id,category,digital
0,PC - Гарнитуры/Наушники,0,PC,0
1,Аксессуары - PS2,1,Аксессуары,0
2,Аксессуары - PS3,2,Аксессуары,0
3,Аксессуары - PS4,3,Аксессуары,0
4,Аксессуары - PSP,4,Аксессуары,0
5,Аксессуары - PSVita,5,Аксессуары,0
6,Аксессуары - XBOX 360,6,Аксессуары,0
7,Аксессуары - XBOX ONE,7,Аксессуары,0
8,Билеты (Цифра),8,Билеты,1
9,Доставка товара,9,Доставка товара,0


In [19]:
def get_category(item_id):
    category_id = df_items.loc[df_items['item_id'] == item_id]['item_category_id'].values[0]
    return df_categories.loc[df_categories['item_category_id'] == category_id]['category'].values[0]

def get_category_digitality(item_id):
    category_id = df_items.loc[df_items['item_id'] == item_id]['item_category_id'].values[0]
    return df_categories.loc[df_categories['item_category_id'] == category_id]['digital'].values[0]

def get_shop_type(shop_id):
    return df_shops.loc[df_shops['shop_id'] == shop_id]['type'].values[0]

def get_shop_megality(shop_id):
    return df_shops.loc[df_shops['shop_id'] == shop_id]['mega'].values[0]


In [20]:
#get_category_digitability(8)
#get_shop_megality(27)
df_sales['shop_id'] = df_sales['shop_id'].apply(lambda x: valid_shop_id(x))

In [23]:
df_monthly_sales = df_sales.groupby(['date_block_num','shop_id','item_id'])['item_cnt_day'].sum().to_frame(name='item_cnt_month').reset_index()

df_monthly_sales.head(10)

,date_block_num,shop_id,item_id,item_cnt_month
0,0,2,27,1.0
1,0,2,33,1.0
2,0,2,317,1.0
3,0,2,438,1.0
4,0,2,471,2.0
5,0,2,481,1.0
6,0,2,482,1.0
7,0,2,484,2.0
8,0,2,491,1.0
9,0,2,534,2.0


In [32]:
%%time
#df_monthly_sales['shop_type'] = df_monthly_sales['shop_id'].apply(lambda x: get_shop_type(x))
df_monthly_sales['shop_type'] = df_monthly_sales['shop_id'].apply(get_shop_type)

CPU times: user 37min 24s, sys: 9.01 s, total: 37min 33s
Wall time: 38min 16s


In [33]:
%%time
df_monthly_sales['shop_mega'] = df_monthly_sales['shop_id'].apply(lambda x: get_shop_megality(x))

CPU times: user 36min 40s, sys: 11 s, total: 36min 51s
Wall time: 37min 28s


In [34]:
%%time
df_monthly_sales['category'] = df_monthly_sales['item_id'].apply(lambda x: get_category(x))

CPU times: user 1h 18min 52s, sys: 801 ms, total: 1h 18min 53s
Wall time: 1h 20min 39s


In [35]:
%%time
df_monthly_sales['digital'] = df_monthly_sales['item_id'].apply(lambda x: get_category_digitality(x))

CPU times: user 1h 19min 6s, sys: 3.81 s, total: 1h 19min 9s
Wall time: 1h 21min


In [49]:
#df_monthly_sales.head(10)
df_monthly_sales['category'].unique()

array(['Игры', 'Кино', 'Книги', 'Программы', 'Игры PC', 'Подарки',
       'Музыка', 'Аксессуары', 'Карты оплаты', 'Игровые консоли',
       'Чистые носители', 'Служебные', 'Элементы питания',
       'Доставка товара', 'PC', 'Билеты', 'Игры MAC', 'Игры Android'],
      dtype=object)

In [37]:
df_monthly_sales.to_csv("monthly_sales.csv", index=False)

In [46]:
df_monthly_sales['month'] = df_monthly_sales['date_block_num']%12 +1

In [56]:
X_train = df_monthly_sales[['shop_type','shop_mega','category','digital','month']].copy()
Y_train = df_monthly_sales['item_cnt_month'].copy()

In [57]:
X_train.head()

,shop_type,shop_mega,category,digital,month
0,ТЦ,1,Игры,0,1
1,ТЦ,1,Кино,0,1
2,ТЦ,1,Книги,0,1
3,ТЦ,1,Книги,0,1
4,ТЦ,1,Книги,0,1


In [53]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score

date_ix = 0

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attributes_names):
        self.attributes_names = attributes_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attributes_names].values # convert to NumPy array

class CycleTransformator( BaseEstimator, TransformerMixin):
    #Class Constructor 
    def __init__( self,  cycle_columns ):
        self._cycle_columns = cycle_columns
        self._cycle_stats = {}
    
    #Return self nothing else to do here    
    def fit( self, X, y = None ):
        for column in self._cycle_columns:
            self._cycle_stats[column] = { 'max': X[column].max(), 'min': X[column].min() }
        return self
    
    #Method that describes what we need this transformer to do
    def transform( self, X, y = None ):
        self._df = pd.DataFrame(index=X.index)
        for column in self._cycle_columns:
            self._df[column+'_sin'] = np.sin(2*np.pi/(self._cycle_stats[column]['max']+1)*X[column].fillna(self._cycle_stats[column]['min']-1))
            self._df[column+'_cos'] = np.cos(2*np.pi/(self._cycle_stats[column]['max']+1)*X[column].fillna(self._cycle_stats[column]['min']-1))

        #self._df = self._df.reset_index(drop=True)
        #print('Cycle transform shape is {}'.format(self._df.values.shape))
        return self._df.values

In [54]:
num_attribs = ['shop_mega','digital']
cat_attribs = ['shop_type', 'category']
date_attribs = ['month']

num_pipeline = Pipeline([
    ('selector', DataFrameSelector(num_attribs)),
    ('imputer', SimpleImputer(strategy="median")),
#    ('std_scaler', StandardScaler()),
])
cat_pipeline = Pipeline([
    ('selector', DataFrameSelector(cat_attribs)),
    ('cat_encoder', OneHotEncoder(sparse=False)),
])
counted_pipeline = Pipeline([
    ('cycle_transformator', CycleTransformator(cycle_columns=date_attribs)),
])

full_pipeline = FeatureUnion(transformer_list=[
    ('num_pipeline', num_pipeline),
    ('cat_pipeline', cat_pipeline),
    ('counted_pipeline', counted_pipeline),
])

In [58]:
X_train_prepared = full_pipeline.fit_transform(X_train)
X_train_prepared.shape

(1606770, 28)

In [60]:
%%time
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Std Deviation:", scores.std())

# train
tree_reg = DecisionTreeRegressor(random_state=57)
tree_reg.fit(X_train_prepared, Y_train)
# predict
predictions = tree_reg.predict(X_train_prepared)

scores = cross_val_score(tree_reg, X_train_prepared, Y_train, scoring="neg_mean_squared_error")
tree_rmse_scores = np.sqrt(-scores)

display_scores(tree_rmse_scores)
#print("Metric: %.2f" % CountScore(Y_train.values, predictions) )
print("R2-score: %.2f" % r2_score(predictions , Y_train.values) )

Scores: [ 4.82319232  8.94688727  8.56996397 17.02793548  9.24683549]
Mean: 9.722962903995946
Std Deviation: 3.9882019325702562
R2-score: -7.57
CPU times: user 1min 14s, sys: 962 ms, total: 1min 15s
Wall time: 1min 15s


In [ ]:
%%time
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(random_state=57)
forest_reg.fit(X_train_prepared, Y_train.values)
# predict
predictions = forest_reg.predict(X_train_prepared)

scores = cross_val_score(forest_reg, X_train_prepared, Y_train.values, scoring="neg_mean_squared_error")
forest_rmse_scores = np.sqrt(-scores)
display_scores(forest_rmse_scores)
#print("Metric: %.2f" % CountScore(Y_train.values, ackAI_predictions) )
print("R2-score: %.2f" % r2_score(predictions , Y_train.values) )